In [ ]:
# importamos las librerias necesarias
import pandas as pd
import faker as fk
import random as rd
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import openpyxl
import sympy as sp

In [ ]:
planetas = {
    "Tierra": 9.8,
    "Marte": 3.7,
    "Júpiter": 24.8,
    "Saturno": 9.0,
    "Urano": 8.7,
    "Neptuno": 11.0
}

In [ ]:
# Para el apartado A
L1 = 1.00
t1 = 0.46
g_a = 2*L1 / (t1**2)
print("La aceleración de la gravedad es: ", g_a)

In [ ]:
# Para el apartado B
L2 = 2.00
t2 = 0.66
g_b = 2*L2 / (t2**2)
print("La aceleración de la gravedad es: ", g_b)

In [ ]:
# creamos un dataframe con los datos
df = pd.DataFrame({
    'Longitud': [L1, L2],
    'Tiempo': [t1, t2],
    'Gravedad': [g_a, g_b],
    'Planeta más cercano': ['Tierra', 'Saturno'],
    '2ndo más cercano': ['Saturno', 'Tierra']},index= ['Escenario A', 'Escenario B'])
df

In [ ]:
def calcular_error(L, t):
    L_simbolo, t_simbolo = sp.symbols('L t')
    g_calculada_simbolo = (2*L_simbolo) / (t_simbolo**2)
    
    derivada_parcial_L = sp.diff(g_calculada_simbolo, L_simbolo)
    derivada_parcial_t = sp.diff(g_calculada_simbolo, t_simbolo)
    
    error_L = 0.01
    error_t = 0.01
    
    error_g_calculada = sp.sqrt((derivada_parcial_L * error_L)**2 + (derivada_parcial_t * error_t)**2)
    
    return float(error_g_calculada.subs({L_simbolo: L, t_simbolo: t}))

    

In [189]:
fake = fk.Faker()

def generar_datos(planetas, L_objetivo, t_objetivo):
    L = round(rd.uniform(L_objetivo - 0.1, L_objetivo + 0.1), 2)
    t = round(rd.uniform(t_objetivo - 0.1, t_objetivo + 0.1), 2)
    g_calculada = round((2*L) / (t**2), 2)

    diferencia_minima = float('inf')
    planeta_mas_cercano = ''
    
    diferencia_minima_2 = float('inf')
    segundo_planeta_mas_cercano = ''

    for planeta, gravedad in planetas.items():
        diferencia = abs(gravedad - g_calculada)
        if diferencia < diferencia_minima:
            diferencia_minima = diferencia
            planeta_mas_cercano = planeta
    
    for planeta, gravedad in planetas.items():
        if planeta != planeta_mas_cercano:
            diferencia = abs(gravedad - g_calculada)
            if diferencia < diferencia_minima_2:
                diferencia_minima_2 = diferencia
                segundo_planeta_mas_cercano = planeta
    
    error_g_calculada = calcular_error(L, t)
    error_porcentaje = (error_g_calculada / g_calculada) * 100
    
    return {
        'Id': fake.uuid4(),
        'Longitud (m)': L,
        'Tiempo (s)': t,
        'Gravedad (m/s^2)': g_calculada,
        'Planeta más cercano': planeta_mas_cercano,
        'Segundo Planeta más cercano': segundo_planeta_mas_cercano,
        'Error Gravedad (m/s^2)': error_g_calculada,
        'Error Porcentaje (%)': error_porcentaje
    }

dataframe_a = pd.DataFrame([generar_datos(planetas, L1, t1) for _ in range(1000)])
dataframe_a



,Id,Longitud (m),Tiempo (s),Gravedad (m/s^2),Planeta más cercano,Segundo Planeta más cercano,Error Gravedad (m/s^2),Error Porcentaje (%)
0,59934bbf-f2b7-4fd6-9ae5-33cd38d3b1f3,1.99,0.71,7.90,Urano,Saturno,0.225913,2.859655
1,056ee112-0055-4f79-8213-5b3971170f95,1.99,0.67,8.87,Saturno,Urano,0.268384,3.025751
2,fc6f8dda-89c1-4e67-beab-0f0f9ae98dcf,2.03,0.64,9.91,Tierra,Saturno,0.313578,3.164262
3,65bedd2a-d047-4848-a413-e154c7806702,1.94,0.73,7.28,Urano,Saturno,0.202977,2.788146
4,164f0e31-eb36-43bb-aa0d-d556433f39da,2.00,0.76,6.93,Urano,Saturno,0.185503,2.676805
...,...,...,...,...,...,...,...,...
995,061a1666-4c9e-4644-b132-5e5008266b40,2.04,0.64,9.96,Tierra,Saturno,0.315086,3.163511
996,146862da-3f99-47bd-b1ed-bc83a0ba4096,1.96,0.64,9.57,Tierra,Saturno,0.303032,3.166479
997,76e1fbb3-63fd-454d-b9bd-932f71d012f6,2.06,0.61,11.07,Neptuno,Tierra,0.366983,3.315117
998,fb8f78cc-b592-4099-a322-b8faeef62681,1.99,0.73,7.47,Urano,Saturno,0.208032,2.784896


In [190]:
dataframe_b = pd.DataFrame([generar_datos(planetas, L2, t2) for _ in range(1000)])
dataframe_b

,Id,Longitud (m),Tiempo (s),Gravedad (m/s^2),Planeta más cercano,Segundo Planeta más cercano,Error Gravedad (m/s^2),Error Porcentaje (%)
0,45da2df6-ccd9-4b22-b4cf-5a99ceb2304c,2.05,0.63,10.33,Tierra,Neptuno,0.331787,3.211880
1,39b444c3-5c8b-4cc5-a9c3-a0324ee513a2,1.94,0.67,8.64,Urano,Saturno,0.261829,3.030428
2,a7922597-b23e-4068-87f1-887cd49162b0,2.05,0.62,10.67,Neptuno,Tierra,0.347975,3.261249
3,458e6fa6-a29e-4e6c-ab30-bd0e85d27535,2.02,0.73,7.58,Urano,Saturno,0.211067,2.784519
4,48cebbad-1f0a-4ea3-848e-328db4741895,1.92,0.60,10.67,Neptuno,Tierra,0.359870,3.372724
...,...,...,...,...,...,...,...,...
995,02bd4fc1-becb-4e36-bbdd-054f3c9b8dce,1.99,0.71,7.90,Urano,Saturno,0.225913,2.859655
996,7a8897ba-7e4a-4238-b021-b1f3ecae9b55,1.92,0.69,8.07,Urano,Saturno,0.237528,2.943342
997,637f1595-1e8b-4c64-a310-58aab600def7,2.05,0.73,7.69,Urano,Saturno,0.214103,2.784171
998,d61e67d2-4ee1-4d4a-b6c3-ffaf06ea38ba,2.01,0.57,12.37,Neptuno,Tierra,0.438484,3.544738


In [ ]:
def contar_planetas(dataframe):
    conteo_planetas = dataframe['Planeta más cercano'].value_counts()
    
    planeta_mas_cercana_nave = conteo_planetas.idxmax()
    
    conteo_planetas_2 = dataframe['2ndo más cercano'].value_counts()
    
    segundo_planeta_mas_cercana_nave = conteo_planetas_2.idxmax()
    
    return conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave


conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave = contar_planetas(dataframe)

print(conteo_planetas)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave}')

In [194]:
# creamos un archivo excel con los datos
def crear_excel_exp_A():
    df = pd.DataFrame(dataframe_a)
    df.to_excel('Datos_experimentos_A.xlsx', index=False)
    print('Archivo A creado con éxito')


def crear_excel_exp_B():
    df = pd.DataFrame(dataframe_b)
    df.to_excel('Datos_experimentos_B.xlsx', index=False)
    print('Archivo  B creado con éxito')

crear_excel_exp_A()
crear_excel_exp_B()


Archivo A creado con éxito
Archivo  B creado con éxito


In [ ]:
print(df.describe())

In [ ]:
# histogramas
df.hist(column=['Longitud (m)', 'Tiempo (s)', 'Gravedad (m/s^2)'])
plt.show()

In [ ]:
# grafico de dispersion
plt.scatter(df['Longitud (m)'], df['Tiempo (s)'])
plt.xlabel('Longitud (m)')
plt.ylabel('Tiempo (s)')
plt.show()

In [ ]:
# codificar etiquetas de planetas 
codificador_etiquetas = LabelEncoder()
df['Planeta más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['Planeta más cercano'])
df['2ndo más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['2ndo más cercano'])

In [ ]:
# correlación entre variables 
df_numeric = df.select_dtypes(include=[np.number])
print(df_numeric.corr())


In [ ]:
# regresion lineal 
X = df['Longitud (m)'].values.reshape(-1, 1)
y = df['Tiempo (s)'].values.reshape(-1, 1)
regresion_lineal = LinearRegression()
regresion_lineal.fit(X, y)

y_pred = regresion_lineal.predict(X)
plt.scatter(X, y, color='blue') 
plt.plot(X, y_pred, color='red')
plt.xlabel('Longitud (m)')
plt.ylabel('Tiempo (s)')
plt.show()

In [ ]:
def contar_planetas(dataframe):
    conteo_planetas = dataframe_a['Planeta más cercano'].value_counts()
    
    planeta_mas_cercana_nave = conteo_planetas.idxmax()
    
    conteo_planetas_2 = dataframe_a['Segundo Planeta más cercano'].value_counts()
    
    segundo_planeta_mas_cercana_nave = conteo_planetas_2.idxmax()
    
    return conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave


conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave = contar_planetas(dataframe)

print(conteo_planetas)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave}')

In [ ]:
def contar_planetas(dataframe):
    conteo_planetas = dataframe_b['Planeta más cercano'].value_counts()
    
    planeta_mas_cercana_nave = conteo_planetas.idxmax()
    
    conteo_planetas_2 = dataframe_b['Segundo Planeta más cercano'].value_counts()
    
    segundo_planeta_mas_cercana_nave = conteo_planetas_2.idxmax()
    
    return conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave


conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave = contar_planetas(dataframe)

print(conteo_planetas)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave}')